## 1. Neural Networks
+ Made using `torch.nn` package
+ `nn` depends on `autograd` to define models and distinguish them.
+ `nn.Module` contains layers, and a method `forward(input)` that returns the `output`.
![convnet](../reports/figures/1_neural_network.png)
+ Training Procedure Neutral Network
    + Define a NN with learnable parameters (or weights)
    + Iterate over dataset of inputs
    + Process input through Network
    + Compute the loss (how far is the output from being correct)
    + Propogate graients back into the network's parameters
    + Update the weights of the network, typically using a simple rule: `weights - learning_rate * gradient`

### **1.1 Define a network**

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    """
    Summary: 
        Defining the forward function and backward function (were the gradients are              computed). This is automatically defined for you using autograd. You can use
        any of the Tensor operations in the forward function.

    """

    def __init__(self):
        
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernal
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120) # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[:1] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            number *= 5
        return num_features


# print out the NN
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [19]:
# net.parameters() are the learnable parameters
params = list(net.parameters())
print("You have {} learnable parameters.".format(len(params)))
print("Size of Conv1's weights {}".format(params[0].size()))

You have 10 learnable parameters.
Size of Conv1's weights torch.Size([6, 1, 3, 3])
